In [37]:
import csv
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import os
from dotenv import load_dotenv
from datetime import datetime
from datetime import date
from datetime import timedelta
import requests
import json

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
import numpy as np

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

# Input Portfolio
Read in the input portfolio CSV file and output a portfolio dataframe
* Inputs:
    * Portfolio CSV file
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares
    * Assumptions: Date unsorted, no '$' sign, no commas
* Outputs:
    * Portfolio database
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares, Transaction Cost
    * Database sorted by ascending date

In [38]:
# Set the path for the CSV file
input_file_path = Path("TestPortfolio.csv")

In [39]:
#Create empty list to store row data from CSV
portfolio_transactions = []

In [40]:
#Open the CSV file; read in row data; calculate transaction cost per transacation
with open(input_file_path, 'r') as csvfile:
    #Set reader object
    csvreader = csv.reader(csvfile, delimiter = ',')
 
    #Read the header
    csv_header = next(csvreader)
    
    #Create new column
    csv_header.append("Transaction Cost")
    portfolio_transactions.append(csv_header)

    #Iterate through the data 
    for row in csvreader:
        #Calculate transaction cost
        cost = float(row[3]) * float(row[4])
        
        #Append transaction cost to list
        row.append(cost)
        portfolio_transactions.append(row)

In [41]:
#Convert transaction list to dataframe
portfolio_df = pd.DataFrame(portfolio_transactions, columns=['Ticker', 'Transaction', 'Date', 'Transaction Price', 'Number of shares', 'Transaction Cost'])
#Drop first row (duplicate header)
portfolio_df=portfolio_df.drop(portfolio_df.index[0])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Transaction Cost
1,AAPL,B,3/5/12,77.8,100,7780
2,AAPL,B,4/1/13,60.57,100,6057
3,TSLA,B,9/23/13,189.9,100,18990
4,TSLA,S,11/18/13,122.38,75,9178.5
5,TSLA,B,5/9/16,208,30,6240
6,GOOG,B,8/7/17,915.39,10,9153.9
7,AAPL,S,3/16/20,240,50,12000


In [42]:
#Convert Date to datetime and set as index
portfolio_df['Date']=pd.to_datetime(portfolio_df['Date'])
portfolio_df.set_index(['Date'], inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


In [43]:
# Sort dataframe in ascending order
portfolio_df.sort_index(inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


# Pull Stock Data
Using the portfolio database, determine the number of unique stocks and pull the stock history.
* Inputs:
    * Portfolio database (portfolio_df)
* Outputs:
    * Ticker Stat database (ticker_stats_df) (Tickers as Rows)
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
    * Dailiy Closing Price database (closing_df)

In [44]:
# Set environment variables from the .env file
# rel_path = Path('../../Python') / '.env'
# load_dotenv(rel_path)
load_dotenv("/Users/heenaroy/Desktop/.env")

True

In [45]:
# Grab API Key for https://financialmodelingprep.com
stock_key = os.getenv("FINANCIAL_MODEL_KEY")

In [46]:
# Create empty list for stock tickers
stock_list=[]

In [47]:
# Determine unique stocks
group=portfolio_df.groupby('Ticker')
# Store unique stocks to stock list
group.groups.keys()
stock_list=list(group.groups)
stock_list

['AAPL', 'GOOG', 'TSLA']

In [48]:
#Create new dataframe resetting date index
portfolio2_df=portfolio_df.reset_index()
portfolio2_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
0,2012-03-05,AAPL,B,77.8,100,7780
1,2013-04-01,AAPL,B,60.57,100,6057
2,2013-09-23,TSLA,B,189.9,100,18990
3,2013-11-18,TSLA,S,122.38,75,9178.5
4,2016-05-09,TSLA,B,208,30,6240
5,2017-08-07,GOOG,B,915.39,10,9153.9
6,2020-03-16,AAPL,S,240,50,12000


In [49]:
#Find the min dates per tikcer
min_stock_dates=portfolio2_df.groupby('Ticker').min()
min_stock_dates

,Date,Transaction,Transaction Price,Number of shares,Transaction Cost
Ticker,,,,,
AAPL,2012-03-05,B,240,100,6057.0
GOOG,2017-08-07,B,915.39,10,9153.9
TSLA,2013-09-23,B,122.38,100,6240.0


In [50]:
#Store yesterday's date
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
yesterday

'2020-06-14'

In [53]:
#Define Main Loop Variables
stock_dict=pd.DataFrame()            # Create dictionary to store stock data
count=0                  # Create counter as index for looping through stock list

#For each ticker in the stock list, pull the stock data
for ticker in stock_list:
    #Define new dictionary key set per ticker
    stock_dict[ticker]={
        'PE Ratio'            : 0,
        'Profit Margin'       : 0,
        'Cash'                : 0,
        'Beta'                : 0,
        'Last Dividend'       : 0,
        'Industry'            : 0,
        'Sector'              : 0,
        'Daily Closing Price' : 0,
        }
    
######
    #Get Beta, Last Div, Industry and Sector
    url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    #Store stock data in dictionary
    stock_dict[ticker]['Beta']=parsed[0]['beta']
    stock_dict[ticker]['Last Dividend']=parsed[0]['lastDiv']
    stock_dict[ticker]['Industry']=parsed[0]['industry']
    stock_dict[ticker]['Sector']=parsed[0]['sector']
    

######
    # Grab PE Ratio
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store PE Ratio in dictionary
    stock_dict[ticker]['PE Ratio']=parsed[0]['peRatio']
    
    
######    
    # Grab Profit Margin
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Profit Margin']=parsed[0]['netProfitMargin']
    
   
###### 
    # Grab Cash
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Cash']=parsed[0]['cashAndCashEquivalents']

     
######
    #Grab the first purchase date for current ticker
    datetime_obj=datetime.strptime(str(min_stock_dates['Date'][ticker]),"%Y-%m-%d %H:%M:%S")
    date_obj=datetime_obj.date()
    first_purchase_date=str(date_obj)
    
    # Grab the Daily Closing Prices between the first purchase date and yesterday
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={first_purchase_date}&to={yesterday}&apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
 
    ### Daily Stock Pull Variables
    count2=0                 # Create counter as index for looping through dictionary of stock data    
    daily_dict={}            # Create empty dictionary to store daily stock data before appending to daily_list_of_dict
    daily_list_of_dict = []  # Create empty list to store dictionary info from daily_dict before storing to stock_dict

    # Loop through historical data and grab daily closing date and price
    for i in parsed['historical']:
        daily_dict['Date'] = parsed['historical'][count2]['date']
        daily_dict['Price'] = parsed['historical'][count2]['close']
        daily_list_of_dict.append(daily_dict.copy())
        count2+=1

    #Store historical stock data to stock_dict dictionary
    stock_dict[ticker]['Daily Closing Price']=daily_list_of_dict
    
    
    #increment counter
    count+=1 

In [54]:
stock_dict.to_csv("/Users/heenaroy/Desktop/Fintech/Project-Show-Me-The-Money/Working Directory/Heena/stock_dict.csv")

In [55]:
#Copy stock_dict so that main loop above doesn't have to be run multiple times
stock_dict_copy = stock_dict
stock_dict_copy

,AAPL,GOOG,TSLA
PE Ratio,20.8135,14.6956,-131.582
Profit Margin,0.212381,0.212181,-0.035072
Cash,48844000000,18498000000,6268000000
Beta,1.2285,1.02276,0.580101
Last Dividend,3.08,0,0
Industry,Consumer Electronics,Internet Content & Information,Auto Manufacturers
Sector,Technology,Communication Services,Consumer Cyclical
Daily Closing Price,"[{'Date': '2020-06-12', 'Price': 338.8}, {'Dat...","[{'Date': '2020-06-12', 'Price': 1413.18}, {'D...","[{'Date': '2020-06-12', 'Price': 935.28}, {'Da..."


In [56]:
# Create DataFrame showing Beta, P/E Ratio, Profit Margin, 
# Cash, and current Dividends per ticker
ticker_stats_df=DataFrame(stock_dict_copy)
ticker_stats_df.drop('Daily Closing Price', inplace=True)
ticker_stats_df

,AAPL,GOOG,TSLA
PE Ratio,20.8135,14.6956,-131.582
Profit Margin,0.212381,0.212181,-0.035072
Cash,48844000000,18498000000,6268000000
Beta,1.2285,1.02276,0.580101
Last Dividend,3.08,0,0
Industry,Consumer Electronics,Internet Content & Information,Auto Manufacturers
Sector,Technology,Communication Services,Consumer Cyclical


In [57]:
# Create a transpose of ticker_stats_df
ticker_stats_df_T=ticker_stats_df.T
ticker_stats_df_T=ticker_stats_df_T[['PE Ratio', 'Profit Margin', 'Cash', 'Beta', 'Last Dividend', 'Industry', 'Sector']]
ticker_stats_df_T

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.8135,0.212381,48844000000,1.2285,3.08,Consumer Electronics,Technology
GOOG,14.6956,0.212181,18498000000,1.02276,0,Internet Content & Information,Communication Services
TSLA,-131.582,-0.035072,6268000000,0.580101,0,Auto Manufacturers,Consumer Cyclical


In [ ]:
# Closing Data

In [58]:
# Create DataFrame showing Daily Closing Price per ticker
daily_closing_df=DataFrame(stock_dict_copy)
daily_closing_df.drop(['Beta', 'Cash', 'Industry', 'Last Dividend', 'PE Ratio', 'Profit Margin', 'Sector'], inplace=True)
daily_closing_df

,AAPL,GOOG,TSLA
Daily Closing Price,"[{'Date': '2020-06-12', 'Price': 338.8}, {'Dat...","[{'Date': '2020-06-12', 'Price': 1413.18}, {'D...","[{'Date': '2020-06-12', 'Price': 935.28}, {'Da..."


In [59]:
aapl_df = pd.DataFrame(daily_closing_df['AAPL'][0])
#aapl_df.rename(columns={'Date' : 'AAPL Closing Date', 'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.rename(columns={'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.head()

,Date,AAPL Closing Price
0,2020-06-12,338.80
1,2020-06-11,335.90
2,2020-06-10,352.84
3,2020-06-09,343.99
4,2020-06-08,333.46


In [60]:
goog_df = pd.DataFrame(daily_closing_df['GOOG'][0])
#goog_df.rename(columns={'Date' : 'GOOG Closing Date', 'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.rename(columns={'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.head()

,Date,GOOG Closing Price
0,2020-06-12,1413.18
1,2020-06-11,1403.84
2,2020-06-10,1465.85
3,2020-06-09,1456.16
4,2020-06-08,1446.61


In [61]:
tsla_df = pd.DataFrame(daily_closing_df['TSLA'][0])
#tsla_df.rename(columns={'Date' : 'TSLA Closing Date', 'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.rename(columns={'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.head()

,Date,TSLA Closing Price
0,2020-06-12,935.28
1,2020-06-11,972.84
2,2020-06-10,1025.05
3,2020-06-09,940.67
4,2020-06-08,949.92


In [62]:
closing_df = aapl_df.set_index('Date').join(goog_df.set_index('Date')).join(tsla_df.set_index('Date'))
closing_df.head()

,AAPL Closing Price,GOOG Closing Price,TSLA Closing Price
Date,,,
2020-06-12,338.80,1413.18,935.28
2020-06-11,335.90,1403.84,972.84
2020-06-10,352.84,1465.85,1025.05
2020-06-09,343.99,1456.16,940.67
2020-06-08,333.46,1446.61,949.92


In [192]:
# for ticker in daily_closing_df:
# #     print(ticker)

    
#     for list_item in daily_closing_df[ticker]:
# #         print(list_item)
# #         print(list_item[0])

#         for row in list_item:
#             print(row)


{'Date': '2020-06-10', 'Price': 352.84}
{'Date': '2020-06-09', 'Price': 343.99}
{'Date': '2020-06-08', 'Price': 333.46}
{'Date': '2020-06-05', 'Price': 331.5}
{'Date': '2020-06-04', 'Price': 322.32}
{'Date': '2020-06-03', 'Price': 325.12}
{'Date': '2020-06-02', 'Price': 323.34}
{'Date': '2020-06-01', 'Price': 321.85}
{'Date': '2020-05-29', 'Price': 317.94}
{'Date': '2020-05-28', 'Price': 318.25}
{'Date': '2020-05-27', 'Price': 318.11}
{'Date': '2020-05-26', 'Price': 316.73}
{'Date': '2020-05-22', 'Price': 318.89}
{'Date': '2020-05-21', 'Price': 316.85}
{'Date': '2020-05-20', 'Price': 319.23}
{'Date': '2020-05-19', 'Price': 313.14}
{'Date': '2020-05-18', 'Price': 314.96}
{'Date': '2020-05-15', 'Price': 307.71}
{'Date': '2020-05-14', 'Price': 309.54}
{'Date': '2020-05-13', 'Price': 307.65}
{'Date': '2020-05-12', 'Price': 311.41}
{'Date': '2020-05-11', 'Price': 315.01}
{'Date': '2020-05-08', 'Price': 310.13}
{'Date': '2020-05-07', 'Price': 303.74}
{'Date': '2020-05-06', 'Price': 300.63}
{

In [166]:

# daily_closing_df3=pd.DataFrame()


# for ticker in daily_closing_df:
#     for list_item in daily_closing_df[ticker]:
# #         print(list_item[0])
# #         count=0
#         for row in list_item:
#             print(row)
# #             daily_closing_df3=daily_closing_df3.append(row)
#             if count==0:
#                 print(type(row['Date']))
#                 daily_closing_df3[f'{ticker} Closing Date']=row['Date']
#                 print(type(daily_closing_df3[f'{ticker} Closing Date']))
#                 daily_closing_df3[f'{ticker} Closing Price']=row['Price']
#             else:
#                 pass
# # #                 daily_closing_df3[f'{ticker} Closing Date'].append(row['Date'])
# #                 daily_closing_df3[f'{ticker} Closing Price'].append(row['Price'])
#             count+=1
        
    
# daily_closing_df3

{'Date': '2020-06-10', 'Price': 352.84}
{'Date': '2020-06-09', 'Price': 343.99}
{'Date': '2020-06-08', 'Price': 333.46}
{'Date': '2020-06-05', 'Price': 331.5}
{'Date': '2020-06-04', 'Price': 322.32}
{'Date': '2020-06-03', 'Price': 325.12}
{'Date': '2020-06-02', 'Price': 323.34}
{'Date': '2020-06-01', 'Price': 321.85}
{'Date': '2020-05-29', 'Price': 317.94}
{'Date': '2020-05-28', 'Price': 318.25}
{'Date': '2020-05-27', 'Price': 318.11}
{'Date': '2020-05-26', 'Price': 316.73}
{'Date': '2020-05-22', 'Price': 318.89}
{'Date': '2020-05-21', 'Price': 316.85}
{'Date': '2020-05-20', 'Price': 319.23}
{'Date': '2020-05-19', 'Price': 313.14}
{'Date': '2020-05-18', 'Price': 314.96}
{'Date': '2020-05-15', 'Price': 307.71}
{'Date': '2020-05-14', 'Price': 309.54}
{'Date': '2020-05-13', 'Price': 307.65}
{'Date': '2020-05-12', 'Price': 311.41}
{'Date': '2020-05-11', 'Price': 315.01}
{'Date': '2020-05-08', 'Price': 310.13}
{'Date': '2020-05-07', 'Price': 303.74}
{'Date': '2020-05-06', 'Price': 300.63}
{

""


In [ ]:
# closing_df.groupby('Closing Date').count()

# Create Ticker Stats Panel
Using the portfolio database, determine the number of unique stocks and pull stock history.
* Inputs:
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
* Outputs:
    * Ticker Stat Panel (stats_panel)

In [194]:
# Define function that will change negative numbers to red text
def color_negative_red(value):
    if value < 0:
        color='red'
    else:
        color='black'
    return 'color: %s' % color

# Define function that will highlight the max value
def highlight_max(value):
    is_max = value == value.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [195]:
#Format ticker_stats_df_T to:
# Highlight negative values red
# Highlight larget value yellow
# Add '$' and ',' to currency
# Change percentages to %
# Change floats to 2 decimal places
formatted_stats_df = ticker_stats_df_T.style.applymap(
    color_negative_red, subset=['Beta', 'Cash', 'PE Ratio', 'Profit Margin']
    ).apply(
    highlight_max, subset=['Beta', 'Cash', 'Last Dividend', 'PE Ratio', 'Profit Margin']
    ).format(
    {'Beta':'{:,.2f}', 'Cash':'${:,}', 'Last Dividend':'${:,.2f}', 'PE Ratio':'{:,.2f}', 'Profit Margin':'{:,.1%}'}
    )
formatted_stats_df

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.81,21.2%,"$48,844,000,000",1.23,$3.08,Computer Hardware,Technology
GOOG,14.70,21.2%,"$18,498,000,000",1.02,$0.00,Online Media,Technology
TSLA,-131.58,-3.5%,"$6,268,000,000",0.58,$0.00,Autos,Consumer Cyclical


In [196]:
stats_panel = pn.panel(formatted_stats_df)
stats_panel

HTML(Styler)

In [ ]:
# code below is my scratchpad area, ignore